In [20]:
import pandas as pd
import glob as gb

In [2]:
# This is being turned off because I am only doing a typecast
# and copy, which doesn't seem to have any side effects.
pd.options.mode.chained_assignment = None  # default='warn'

def tsv_to_df(filename: str, column_lookup: dict):
    with open(filename, "r") as f:
        tsv_contents = f.read() 
    tsv_contents = tsv_contents.split("\n")
    reformatted_tsv_contents = [line.split() for line in tsv_contents]
    base_name = filename.split(".")[0]
    csv_filename = base_name + ".csv"
    column_names = column_lookup[base_name]
    return pd.DataFrame(reformatted_tsv_contents, columns=column_names)


def is_number_sequence(sequence: list) -> bool:
    sequence.sort()
    start = sequence[0]
    end = sequence[-1]
    return list(range(start, end+1)) == sequence


def understanding_customer_ids(dfs: list):
    customer_ids = []
    for filename, df in dfs.items():
        customer_ids = df["CustomerID"]
        customer_ids = set(customer_ids)
        print("Filename: ", filename)
        print("Number of customer IDs across all tsv's:", len(customer_ids))
        customer_ids = [int(elem) for elem in customer_ids if pd.notnull(elem)]
        customer_ids.sort()
        if is_number_sequence(customer_ids):
            print("And the customer id's are an inclusive integer sequence")
        else:
            print("And the customer id's are not an inclusive integer sequence")

##What does this do? 
def segment_for_customer_id(dfs: list):
    nulls_removed = []
    for df in dfs:
        tmp_df = df[df["CustomerID"].notnull()]
        tmp_df["CustomerID"] = tmp_df["CustomerID"].astype(int)
        nulls_removed.append(tmp_df)
    values_removed = []
    for df in nulls_removed:
        values_removed.append(df[df["CustomerID"] < 750])
    return values_removed


def generate_final_df(dfs: list):
    final_df = dfs[0]
    for df in dfs[1:]:
        final_df = pd.merge(final_df, df, on="CustomerID")
    final_df.to_csv("ds-all-data.csv", index=False)



#     uncomment this function to understand more about
#     customer IDs - they are not the same across all
#     the csv files
    filename_dfs = {csv_file:pd.read_csv(csv_file) ##ERIC - why make this part into a dict
                    for csv_file in glob("*.csv")}
    understanding_customer_ids(filename_dfs)

In [25]:
filename_dfs = {csv_file:pd.read_csv(csv_file)
                    for csv_file in gb.glob("*.csv")}
understanding_customer_ids(filename_dfs)

Filename:  ds-all-data.csv
Number of customer IDs across all tsv's: 514
And the customer id's are not an inclusive integer sequence


In [ ]:
def understanding_customer_ids(dfs: list):
    customer_ids = []
    for filename, df in dfs.items():
        customer_ids = df["CustomerID"]
        customer_ids = set(customer_ids)
        print("Filename: ", filename)
        print("Number of customer IDs across all tsv's:", len(customer_ids))
        customer_ids = [int(elem) for elem in customer_ids if pd.notnull(elem)] ##ERIC - does this part just create a list of the non null values
        customer_ids.sort()
        if is_number_sequence(customer_ids):
            print("And the customer id's are an inclusive integer sequence")
        else:
            print("And the customer id's are not an inclusive integer sequence")

In [27]:
filename_dfs.items()

dict_items([('ds-all-data.csv',      CustomerID CheckingAccountBalance DebtsPaid SavingsAccountBalance  \
0           345                   some      paid                  none   
1           243                   some   delayed                  some   
2           662                   debt   delayed                  some   
3           102                   none      paid                  high   
4           722                   none   delayed                  some   
5           316                   none      paid                  none   
6           236                   none   delayed                  none   
7           572                   debt      paid                  some   
8           436                   none   delayed                medium   
9           338                   none   delayed                  none   
10          499                   debt      paid                  some   
11          570                   none   delayed                  none   
12    

In [21]:
gb.glob("*.csv")

['ds-all-data.csv']

In [3]:
##RUnning the above functions

if __name__ == '__main__':
    column_lookup = {
        'ds-credit': 'CustomerID CheckingAccountBalance DebtsPaid SavingsAccountBalance CurrentOpenLoanApplications'.split(),
        'ds-app': 'CustomerID LoanPayoffPeriodInMonths LoanReason RequestedAmount InterestRate Co-Applicant'.split(),
        'ds-borrower': 'CustomerID YearsAtCurrentEmployer YearsInCurrentResidence Age RentOrOwnHome TypeOfCurrentEmployment NumberOfDependantsIncludingSelf'.split(),
        'ds-result': 'CustomerID WasTheLoanApproved'.split()
    }
    dfs = []
    for ind, base_name in enumerate(column_lookup):

        filename = base_name + ".tsv"
##ERic   - how do i use debugging tools to see what tsv_to_df(filename, column_lookup) upon one iteration would look like?      
        dfs.append(tsv_to_df(filename, column_lookup))

    dfs = segment_for_customer_id(dfs)
    generate_final_df(dfs)

In [11]:
for ind, base_name in enumerate(column_lookup):
        filename = base_name + ".tsv"
        print(ind,base_name)

0 ds-credit
1 ds-app
2 ds-result
3 ds-borrower


In [10]:
enumerate(column_lookup) #enumerate provides the index and the key val for the dict? correct?

In [ ]:
def tsv_to_df(filename: str, column_lookup: dict):
    with open(filename, "r") as f:
        tsv_contents = f.read() 
    tsv_contents = tsv_contents.split("\n")
    reformatted_tsv_contents = [line.split() for line in tsv_contents]
    base_name = filename.split(".")[0]
    csv_filename = base_name + ".csv"
    column_names = column_lookup[base_name]
    return pd.DataFrame(reformatted_tsv_contents, columns=column_names)

In [13]:
filename = 'ds-credit.tsv'
column_lookup = {'ds-credit': 'CustomerID CheckingAccountBalance DebtsPaid SavingsAccountBalance CurrentOpenLoanApplications'.split()}

In [15]:
tsv_to_df(filename,column_lookup)

,CustomerID,CheckingAccountBalance,DebtsPaid,SavingsAccountBalance,CurrentOpenLoanApplications
0,374,debt,paid,some,2
1,346,some,delayed,none,1
2,345,some,paid,none,1
3,243,some,delayed,some,1
4,662,debt,delayed,some,2
5,102,none,paid,high,1
6,722,none,delayed,some,2
7,316,none,paid,none,1
8,236,none,delayed,none,2
9,572,debt,paid,some,1


## lets do some aggregation things here

In [29]:
df = pd.read_csv('ds-all-data.csv')
df.head()

,CustomerID,CheckingAccountBalance,DebtsPaid,SavingsAccountBalance,CurrentOpenLoanApplications,LoanPayoffPeriodInMonths,LoanReason,RequestedAmount,InterestRate,Co-Applicant,WasTheLoanApproved,YearsAtCurrentEmployer,YearsInCurrentResidence,Age,RentOrOwnHome,TypeOfCurrentEmployment,NumberOfDependantsIncludingSelf
0,345,some,paid,none,1,24,auto,2760,4,none,Y,10+,4,36,free,skilled,1
1,243,some,delayed,some,1,12,goods,1424,4,none,Y,7,3,26,owned,skilled,1
2,662,debt,delayed,some,2,20,goods,4272,1,none,Y,10+,4,24,owned,skilled,1
3,102,none,paid,high,1,6,goods,2978,1,none,Y,4,2,32,owned,skilled,1
4,722,none,delayed,some,2,36,auto,6614,4,none,Y,10+,4,34,owned,self,1


In [ ]:
#say, number of unique customers whose checkingaccountbalance is debt
#stuff like that